## Final Project Part B
**Group:7** <br>
**Members: Chi-Hao Tseng, Hanyu Li, Raghul Balamurugan, Shubhangkar Girish Jain, Tanvi Mehra**<br>
**Course: BUDT 758I**<br>
**Date: 12/10/22**<br>

### Objective: Construct a high-Sharpe portfolio of 8 chosen instruments.
- Construct an minimum vol portfolio of the chosen instruments from Part (A), calculate its Sharpe as our active portfolio.
- Construct an equal-weighted portfolio of the chosen instruments from Part (A), calculate its Sharpe as our passive portfolio and compare it with the active portfolio.
- Comment on the result of the active portfolio and compare its performance with that from Part (A), and give reasons of the differences.

### Imports and Function Definitions

In [1]:
# import libraries
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import scipy.stats as scs
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
import numpy as np
import pandas as pd
import scipy.stats as scs
from sklearn.linear_model import LinearRegression
%matplotlib inline


# universal plot dimensions we will use as the figsize argument for all plots
aspect_ratio=(8,5)
pct_format_4d = '{:.4%}'
pct_format_2d = '{:.2%}'
annualization_factor=260

In [2]:
# data frame "pretty print" function to print percentages. There are some optional parameters
def print_data_frame_pct (dF, pct_format=pct_format_4d, nan='NaN', indexSlice=pd.IndexSlice[:,:]):
    display(dF.style.format(pct_format, na_rep=nan, subset=indexSlice))

# calculate (and print) the summary statistics table
def calc_summary_stats(dF, af=annualization_factor, pct_format=pct_format_2d):
    cumRtns = dF.cumsum()
    summary_stats = pd.DataFrame([af*dF.mean(),
                                  np.sqrt(af)*dF.std(),
                                  np.sqrt(af) * dF.mean() / dF.std(),
                                  (cumRtns - np.maximum.accumulate(cumRtns)).min(axis=0),
                                  (dF >= 0).sum() / dF.shape[0]
                                 ],
                                 index=['Average Annual Return', 'Annual Risk', 'Sharpe Ratio', 'maxDD', 'Success Ratio']
                                )
    print_data_frame_pct(summary_stats, pct_format=pct_format, indexSlice=pd.IndexSlice[['Average Annual Return', 'Annual Risk', 'maxDD', 'Success Ratio'], :])
    #return summary_stats

def read_data_frame(filename, index_col=0, parse_dates=True):
    dF = pd.read_csv(filename, index_col=index_col, parse_dates=parse_dates)
    print('\n1)','INFO')
    dF.info() # print info
    print('\n2)', 'ANY NULL VALUES?')
    print(dF.isnull().sum()) # check if any null values
    print('\n3)', 'SUMMARY STATISTICS')
    display(dF.describe()) # print summary statistics
    display(dF) # eyeball the full dF and then return it
    return dF


In [3]:
df = read_data_frame('PricesThru2022.csv', index_col=[0])


1) INFO
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5704 entries, 1999-12-31 to 2022-10-31
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    5704 non-null   float64
 1   AMZN    5704 non-null   float64
 2   ATT     5704 non-null   float64
 3   AUD     5704 non-null   float64
 4   BOND    5704 non-null   float64
 5   EUR     5704 non-null   float64
 6   GE      5704 non-null   float64
 7   GOLD    5704 non-null   float64
 8   INTC    5704 non-null   float64
 9   SILVER  5704 non-null   float64
 10  SPY     5704 non-null   float64
dtypes: float64(11)
memory usage: 534.8 KB

2) ANY NULL VALUES?
AAPL      0
AMZN      0
ATT       0
AUD       0
BOND      0
EUR       0
GE        0
GOLD      0
INTC      0
SILVER    0
SPY       0
dtype: int64

3) SUMMARY STATISTICS


,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
count,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000
mean,29.123331,33.872244,24.422850,0.772048,1007.537821,1.199985,194.617295,1041.688110,32.077770,15.884630,185.878530
std,42.244986,49.339228,4.996773,0.141465,74.627453,0.159757,87.849497,519.733122,13.413449,8.480358,94.915076
min,0.234286,0.298500,14.607251,0.479000,790.241628,0.827200,43.919998,255.550000,12.080000,4.060000,68.110001
25%,1.821964,2.188375,20.088745,0.696100,960.575137,1.106500,118.307693,456.650000,21.690001,7.610000,118.374998
50%,12.517857,9.222000,24.229607,0.755500,1016.167110,1.200350,197.923080,1191.470000,27.399999,16.063500,142.794998
75%,33.890624,42.209750,27.977718,0.875525,1065.936237,1.318050,255.557697,1390.542500,37.750000,19.821700,232.937503
max,182.009995,186.570496,44.184292,1.102000,1128.970002,1.599100,461.538452,2063.540000,74.875000,48.443800,477.709991


,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
1999-12-31,0.917969,3.806250,36.820244,0.6567,798.154631,1.0062,396.794861,288.00,41.156250,5.3800,146.875000
2000-01-03,0.999442,4.468750,35.498489,0.6585,793.868670,1.0243,384.615387,289.00,43.500000,5.4050,145.437500
2000-01-04,0.915179,4.096875,33.421452,0.6550,796.761677,1.0296,369.230774,282.25,41.468750,5.3250,139.750000
2000-01-05,0.928571,3.487500,33.940708,0.6579,793.118215,1.0321,368.589752,280.00,41.812500,5.1450,140.000000
2000-01-06,0.848214,3.278125,33.043808,0.6533,795.091984,1.0328,373.517639,281.10,39.375000,5.1200,137.750000
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,152.339996,120.599998,17.690001,0.6394,844.938544,0.9966,73.000000,1653.17,27.410000,19.3473,384.920013
2022-10-26,149.350006,115.660004,18.139999,0.6497,849.013953,1.0081,75.459999,1664.57,27.209999,19.5815,382.019989
2022-10-27,144.800003,110.959999,18.030001,0.6452,853.391436,0.9964,76.000000,1663.31,26.270000,19.6014,379.980011
2022-10-28,155.740005,103.410004,18.480000,0.6411,851.238239,0.9965,78.330002,1644.86,29.070000,19.2595,389.019989


In [4]:
df_log = np.log(df / df.shift(1))
df_log.iloc[0] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
df_log.describe()

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
count,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000,5704.000000
mean,0.000897,0.000577,-0.000123,-0.000005,0.000011,-0.000003,-0.000286,0.000304,-0.000065,0.000223,0.000169
std,0.026054,0.031684,0.016574,0.007918,0.002480,0.006066,0.021278,0.010809,0.023779,0.019013,0.012555
min,-0.731248,-0.284568,-0.135375,-0.072937,-0.021011,-0.027742,-0.164394,-0.095121,-0.248885,-0.203851,-0.115887
25%,-0.010417,-0.012075,-0.007404,-0.004117,-0.001367,-0.003474,-0.008932,-0.004970,-0.010578,-0.007705,-0.004942
50%,0.000887,0.000488,0.000291,0.000292,0.000127,0.000073,-0.000202,0.000458,0.000338,0.000797,0.000597
75%,0.013148,0.013546,0.007452,0.004385,0.001467,0.003417,0.008738,0.006040,0.011171,0.009453,0.005977
max,0.128269,0.296181,0.173961,0.080330,0.014643,0.034506,0.179844,0.102451,0.183345,0.131802,0.120666


In [5]:
calc_summary_stats(df_log)

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
Average Annual Return,23.33%,15.01%,-3.20%,-0.12%,0.27%,-0.08%,-7.43%,7.91%,-1.69%,5.79%,4.41%
Annual Risk,42.01%,51.09%,26.72%,12.77%,4.00%,9.78%,34.31%,17.43%,38.34%,30.66%,20.24%
Sharpe Ratio,0.555344,0.293774,-0.119904,-0.009252,0.068464,-0.008413,-0.216442,0.453894,-0.043978,0.188878,0.217679
maxDD,-170.38%,-270.61%,-110.69%,-65.17%,-29.92%,-51.09%,-235.22%,-59.07%,-182.43%,-139.71%,-83.18%
Success Ratio,52.56%,51.42%,51.82%,52.51%,52.26%,50.81%,49.98%,52.73%,51.44%,53.65%,54.19%


### Creating Benchmark DataFrame for Out Of Sample Period

In [6]:
benchmark_df = df_log[['AAPL','AMZN','ATT','INTC','SPY','BOND','GOLD','EUR']].loc['2019-01-01':]
benchmark_df.describe()

,AAPL,AMZN,ATT,INTC,SPY,BOND,GOLD,EUR
count,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000,959.000000
mean,0.001416,0.000324,-0.000175,-0.000523,0.000454,-0.000174,0.000252,-0.000155
std,0.021649,0.022197,0.016901,0.024813,0.014327,0.003133,0.009548,0.004529
min,-0.137708,-0.151398,-0.096963,-0.198957,-0.115887,-0.021011,-0.058632,-0.020642
25%,-0.008606,-0.010708,-0.007545,-0.010605,-0.005152,-0.001616,-0.004566,-0.002758
50%,0.001509,0.001174,0.000345,0.000000,0.001040,0.000073,0.000793,-0.000083
75%,0.013385,0.011248,0.006946,0.011390,0.007495,0.001554,0.005590,0.002532
max,0.113157,0.126949,0.095513,0.178324,0.086731,0.014643,0.049666,0.016152


### Creating Active Portfolio DataFrame

In [7]:
def movingAverageCrossover(dataFrame, fastWindow, slowWindow, flatOrShort):
    if flatOrShort not in (0, -1):
        raise ValueError('0 for flat, and -1 for short only')
    if fastWindow >= slowWindow:
        raise ValueError('fastWindow should be smaller than slowWindow')
    
    df_ma = dataFrame.copy()
    df_ma_fast = df_ma.rolling(fastWindow).mean()
    df_ma_slow = df_ma.rolling(slowWindow).mean()
    df_ma_position = pd.DataFrame(np.where(
        df_ma_fast > df_ma_slow,
        1,
        np.where(df_ma_fast < df_ma_slow, flatOrShort, 0)), columns=df_ma.columns, index=df_ma.index)
    return df_ma_position

In [8]:
# Bollinger band
df_bb = df.copy()
df_bb_mean = df_bb.rolling(20).mean()
df_bb_std = df_bb.rolling(20).std()
df_bb_upper = df_bb_mean + 2 * df_bb_std
df_bb_lower = df_bb_mean - 2 * df_bb_std

df_bb = df_bb[19:]
df_bb_mean = df_bb_mean[19:]
df_bb_std = df_bb_std[19:]
df_bb_upper = df_bb_upper[19:]
df_bb_lower = df_bb_lower[19:]

In [9]:
df_bb_position = pd.DataFrame(0, columns=df_bb.columns, index=df_bb.index)
df_bb_position.iloc[0].loc[df_bb.iloc[0] >= df_bb_upper.iloc[0]] = -1
df_bb_position.iloc[0].loc[df_bb.iloc[0] <= df_bb_lower.iloc[0]] = 1
df_bb_position

,AAPL,AMZN,ATT,AUD,BOND,EUR,GE,GOLD,INTC,SILVER,SPY
2000-01-28,0,0,0,1,0,1,1,0,0,0,1
2000-01-31,0,0,0,0,0,0,0,0,0,0,0
2000-02-01,0,0,0,0,0,0,0,0,0,0,0
2000-02-02,0,0,0,0,0,0,0,0,0,0,0
2000-02-03,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0,0,0,0,0,0,0,0,0,0,0
2022-10-26,0,0,0,0,0,0,0,0,0,0,0
2022-10-27,0,0,0,0,0,0,0,0,0,0,0
2022-10-28,0,0,0,0,0,0,0,0,0,0,0


In [10]:
for i in range(1, len(df_bb)):
    df_bb_position.iloc[i] = df_bb_position.iloc[i-1]
    df_bb_position.iloc[i].loc[df_bb.iloc[i] >= df_bb_upper.iloc[i]] = -1
    df_bb_position.iloc[i].loc[df_bb.iloc[i] <= df_bb_lower.iloc[i]] = 1
    df_bb_position.iloc[i].loc[(df_bb_position.iloc[i-1] == -1) & (df_bb.iloc[i] <= df_bb_mean.iloc[i])] = 0
    df_bb_position.iloc[i].loc[(df_bb_position.iloc[i-1] == 1) & (df_bb.iloc[i] >= df_bb_mean.iloc[i])] = 0

In [11]:
df_bb_return = df_bb_position.shift(1) * df_log
pd.DataFrame(np.sqrt(annualization_factor) * df_bb_return.mean()/df_bb_return.std(), columns=['Sharpe'])

,Sharpe
AAPL,-0.448799
AMZN,0.270114
ATT,0.210601
AUD,-0.015582
BOND,-0.345450
EUR,-0.083902
GE,-0.233101
GOLD,0.128697
INTC,0.339347
SILVER,-0.151921


In [12]:
# recreate the best MA
df_position = movingAverageCrossover(df, 11, 220, 0)
df_ma_return = df_position.shift(1) * df_log

In [13]:
# select the date from 2019-01-01
df_portfolio_ma = df_ma_return[['AAPL', 'AMZN', 'EUR', 'SPY', 'BOND']].loc['2019-01-01':]
df_portfolio_bb = df_bb_return[['ATT', 'INTC']].loc['2019-01-01':]
df_portfolio_self = df_log['GOLD'].loc['2019-01-01':]

In [14]:
# Our selection
df_active_portfolio = pd.merge(pd.merge(df_portfolio_ma, df_portfolio_bb, right_index=True, left_index=True),
    df_portfolio_self, right_index=True, left_index=True)
calc_summary_stats(df_active_portfolio)

,AAPL,AMZN,EUR,SPY,BOND,ATT,INTC,GOLD
Average Annual Return,24.89%,8.77%,0.16%,8.12%,2.08%,3.11%,15.12%,6.56%
Annual Risk,30.94%,24.38%,3.69%,14.61%,3.00%,21.50%,32.51%,15.40%
Sharpe Ratio,0.804234,0.359767,0.042542,0.555505,0.691969,0.144550,0.465202,0.426097
maxDD,-37.73%,-26.37%,-5.89%,-21.01%,-6.62%,-32.30%,-61.02%,-24.05%
Success Ratio,62.04%,69.03%,86.24%,67.67%,77.58%,72.89%,74.04%,54.12%


### Summary Statistics for Active Portfolio in Out of Sample Period

In [15]:
# Mvo weights obtained from part A of the project
mvo_w = [0.02228058, 0.04103796, 0.22833789, 0.18335454, 0.28767587, 0.07081066, 0.04153957, 0.12496294]
mvo_w = np.array(mvo_w)
active_port = pd.DataFrame((mvo_w*df_active_portfolio).sum(axis=1),columns=['active port'])
calc_summary_stats(active_port)

,active port
Average Annual Return,4.70%
Annual Risk,5.56%
Sharpe Ratio,0.846146
maxDD,-9.67%
Success Ratio,55.47%


### Summary Statistics for Benchmark Portfolio(Passive) in Out of Sample Period

In [16]:
# Generating an equal weight array
weights = [0.125]*8
weights = np.array(weights)
equ_wtd_port = pd.DataFrame((weights*benchmark_df).sum(axis=1),columns=['equal weighted port'])
calc_summary_stats(equ_wtd_port)

,equal weighted port
Average Annual Return,4.61%
Annual Risk,16.44%
Sharpe Ratio,0.280699
maxDD,-30.93%
Success Ratio,54.12%


**Yes the active portfolio definitely outperforms the passive portfolio. For the active portfolio the sharpe we achieved is 0.85 while for the passive portfolio the sharpe is 0.2. This behaviour is very similar to the outperformance of benchmark porfolio by the active portfolio in question 11(A) of project part A. We can confidently say that the stratergy we applied works well for the out of sample data as well. Though our strategy outperforms the benchmark, it's sharpe is not as good as the one we have in Part A. We would like to conclude this by the "recession" started from January 2022. Two of our instruments EUR and SPY droped during this period by more than 20% and around 30% respectively. According to this terrible purge, we believe that our stratey can stay with 0.85 in sharpe ratio is quite successful.** 

**"We pledge on our honor that we have not given nor received any unauthorized assistance on this assignment with other groups."**
- Chi-Hao Tseng, Hanyu Li, Raghul Pandi, Shubhangkar Girish Jain, Tanvi Mehra.